# Tutorial 8: Gene imputation

In [ ]:
"""🚀 修复的主训练函数"""
parser = ArgumentParser()
parser = Trainer.add_argparse_args(parser)
parser = OptimizedFixedReferenceBasedGimVI.add_model_specific_args(parser)
parser.add_argument("--ckpt_path", type=str, default=None)

args = parser.parse_args()
args.deterministic = False
args.max_epochs = args.epochs

# GPU配置
print("=== GPU Configuration ===")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")
    print(f"CUDA device name: {torch.cuda.get_device_name()}")
    
    if not hasattr(args, 'accelerator') or args.accelerator is None:
        args.accelerator = "gpu"
    if not hasattr(args, 'devices') or args.devices is None:
        args.devices = 1
        
    print(f"Trainer will use: accelerator={args.accelerator}, devices={args.devices}")
else:
    print("CUDA not available, using CPU")
    args.accelerator = "cpu"
    args.devices = 1
print("========================")

# 设置随机种子
seed_everything(args.seed)

# 应用dataloader优化
print("🔧 Applying dataloader optimizations...")
args = optimize_dataloader_args(args)

# 数据路径
if args.dataset_name == 'mop1_filtered':
    adata_path = f'/media/dang/Omics/omics/baseline/imputation/cell_embeddings_with_GT/processed_adata_mop1_filtered_42_{args.data_pct}.h5ad'
    adata_path2 = None
elif args.dataset_name == 'seqfish':
    adata_path = f'/media/dang/Omics/omics/baseline/imputation/cell_embeddings_with_GT/processed_adata_{args.dataset_name}_{args.seed}_{args.data_pct}.h5ad'
    adata_path2 = None
elif args.dataset_name == 'AD_9494_9498':
    adata_path = f'/media/dang/Omics/omics/baseline/imputation/AD_cell_embeddings_pct/9494_adata_{args.data_pct}pct.h5ad'
    adata_path2 = f'/media/dang/Omics/omics/baseline/imputation/AD_cell_embeddings_pct/9498_adata_{args.data_pct}pct.h5ad'

# 数据模块
datamodule = QueRefCellDataModule(
    adata_path=adata_path,
    adata_path2=adata_path2,
    fold_name=args.fold,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    normalize_embeddings=args.normalize_embeddings,
    log_transform_counts=args.log_transform_counts,
    pin_memory=args.pin_memory,
    reference_ratio=args.reference_ratio,
    k_neighbors=50,
    reference_split_seed=args.reference_split_seed
)

# 设置数据模块
datamodule.setup()
datamodule.dataset_name = args.dataset_name
datamodule.seed = args.seed
datamodule.data_pct = args.data_pct

# 获取数据维度
dims = datamodule.get_dimensions()

# 更新模型参数
args.embedding_dim = dims['embedding_dim']
args.n_available_genes = dims['n_available_genes']
args.n_target_genes = dims['n_target_genes']
args.spatial_dim = dims['spatial_dim']

print(f"📏 Optimized Reference gimVI dimensions:")
print(f"  embedding_dim: {args.embedding_dim}")
print(f"  n_available_genes: {args.n_available_genes}")
print(f"  n_target_genes: {args.n_target_genes}")
print(f"🚀 Optimization parameters:")
print(f"  Train metric frequency: {args.metric_compute_frequency}")
print(f"  Val metric frequency: {args.val_metric_compute_frequency}")
print(f"  Memory cleanup frequency: {args.memory_cleanup_frequency}")
print(f"  Log frequency: {args.log_frequency}")
print(f"  Fast loss: {args.enable_fast_loss}")

# 创建模型
if args.ckpt_path:
    model = OptimizedFixedReferenceBasedGimVI.load_from_checkpoint(args.ckpt_path, **args.__dict__)
else:
    model = OptimizedFixedReferenceBasedGimVI(**args.__dict__)

# 设置callbacks和logger
now = datetime.datetime.now(tz.tzlocal())
extension = now.strftime("%Y_%m_%d_%H_%M_%S")
ckpt_dir = os.path.join(
    BASE_DIR, f"../../../data/ckpts/optimized_reference_gimvi/{args.experiment_name}_{args.dataset_name}_{extension}")
os.makedirs(ckpt_dir, exist_ok=True)

# 🔥 修复callbacks - 使用val_loss而不是val_pearson作为主要监控指标
callbacks = [
    LearningRateMonitor(logging_interval="step"),
    ModelCheckpoint(
        monitor="val_loss",  # 改为val_loss，更稳定
        dirpath=ckpt_dir,
        save_last=True, 
        mode="min",  # 改为min模式
        save_top_k=1
    ),
    EarlyStopping(
        monitor="val_loss",  # 改为val_loss
        min_delta=0.001,
        patience=20,
        verbose=False, 
        mode="min"  # 改为min模式
    )
]

logger_dir = os.path.join(BASE_DIR, f"../../../data")
os.makedirs(logger_dir, exist_ok=True)
wandb_logger = WandbLogger(
    project=f"Optimized_Reference_Based_gimVI", 
    save_dir=logger_dir, 
    name=f"{args.experiment_name}_{args.dataset_name}_{args.data_pct}_{args.fold}_{extension}"
)

# 优化的trainer配置
trainer = Trainer.from_argparse_args(
    args=args,
    deterministic=False,
    callbacks=callbacks,
    logger=wandb_logger,
    precision=32,
    gradient_clip_val=1.0,
    # 添加这些优化参数
    enable_progress_bar=True,
    log_every_n_steps=args.log_frequency * 5,  # 减少logging频率
    # flush_logs_every_n_steps=args.log_frequency * 10,  # 减少log刷新频率
    enable_model_summary=False,  # 禁用模型摘要
    sync_batchnorm=False,  # 如果单GPU，禁用同步BN
    detect_anomaly=False,  # 禁用anomaly detection
)

# 设置trainer的元数据
trainer.dataset_name = args.dataset_name
trainer.seed = args.seed
trainer.data_pct = args.data_pct

if args.mode == "train_test":
    # 训练
    print("🧑‍🏫 Training Optimized Reference-based gimVI...")
    trainer.fit(model, datamodule=datamodule)
    
    if is_save_ckpt:
        best_ckpt_path = os.path.join(ckpt_dir, "best_ckpts.yaml")
        callbacks[1].to_yaml(filepath=best_ckpt_path)
    
    # 加载最佳模型进行测试
    args.ckpt_path = callbacks[1].best_model_path
    if args.ckpt_path:
        model = OptimizedFixedReferenceBasedGimVI.load_from_checkpoint(args.ckpt_path, **args.__dict__)

# 测试
print("🎯 Testing Optimized Reference-based gimVI...")
test_results = trainer.test(model, datamodule=datamodule)
print("✅ Optimized Reference-based gimVI experiment completed!")
print(f"Test results: {test_results}")
